In [1]:
import __main__
__main__.pymol_argv = [ 'pymol', '-q' ]

import pymol
import sys, time, os
import pandas as pd

from pymol import cmd
from pymol import stored
pymol.finish_launching()

In [2]:
cmd.set('fetch_path', cmd.exp_path(r"/home/mangus/Anubis/pdb"), quiet=0)


In [3]:
output = open("Results_34.txt", "w")
output.write("PDB" + "\t" + "No. a.a." + "\t" + "Glycine" + "\t" + "S.S." + "\t" + "Long SS" + "\t" + "Charge" + "\t"+ "SASA" + "\t" + "No. pos." + "\t" + "No. Surf. pos." + "\t" + "Pos. area" + "\t" + "No. neg." + "\t" + "No. Surf. neg." + "\t" + "Neg. area" + "\t" + "No. hyd." + "\t" + "No. Surf. hyd." + "\t" + "hyd. area" + "\t" + "Alpha" + "\t" + "Beta" + "\t" + "Salt bridges" + "\t" + "H-bonds" + "\n")
 #Load PDB entries


180

In [4]:
filename = 'pdb_ids.txt'
# Initialize an empty list to store PDB IDs
PDBs = []

# Open and read the file
with open(filename, 'r') as file:
    for line in file:
        # Strip whitespace and add to the list
        pdb_id = line.strip()
        PDBs.append(pdb_id)

In [5]:
# Load scripts
cmd.do('''run removealt.py''')
cmd.do('''run findSurfaceResidues.py''')
cmd.do('''run get_raw_distances.py''')

In [6]:
for pdb in PDBs: 
	
	# Load protein
	cmd.fetch(pdb)

	# Remove hetatm, all but chain A, and alternative locations
	cmd.remove('hetatm')
	cmd.remove('not (chain A or segment A)')
	cmd.do('''removealt''') 

	# Number of amino acids 
	no_aa = cmd.select('name ca') 
	no_aa = str(no_aa) 

	# Number of glycines 
	no_gly = cmd.select('resname GLY and name ca')
	no_gly = str(no_gly)

	# Calculate SASA
	cmd.do('''set dot_solvent, 1''')
	cmd.do('''set dot_density, 2''')
	sasa = cmd.get_area()
	sasa = str(sasa) 

	cmd.do('''findSurfaceResidues''')

	# Find positively charged residues 
	no_pos = cmd.select('resname ARG+LYS and name ca')
	surf_pos = cmd.select('(byres(exposed_res_01 and resname ARG+LYS)) and name ca')
	surf_pos = str(surf_pos)
	surf_pos_all = cmd.get_area('byres(exposed_res_01 and resname ARG+LYS)')
	surf_pos_area = str(surf_pos_all)

	# Find negatively charged residues 
	no_neg = cmd.select('resname GLU+ASP and name ca')
	surf_neg = cmd.select('(byres(exposed_res_01 and resname GLU+ASP)) and name ca')
	surf_neg = str(surf_neg)
	surf_neg_all = cmd.get_area('byres(exposed_res_01 and resname GLU+ASP)')
	surf_neg_area = str(surf_neg_all)

	# Find hydrophobic residues 
	no_hyd = cmd.select('resname ALA+VAL+ILE+LEU+MET+PHE+TYR+TRP and name ca')
	no_hyd = str(no_hyd) 
	surf_hyd = cmd.select('(byres(exposed_res_01 and resname ALA+VAL+ILE+LEU+MET+PHE+TYR+TRP)) and name ca')
	surf_hyd = str(surf_hyd)
	surf_hyd_all = cmd.get_area('byres(exposed_res_01 and resname ALA+VAL+ILE+LEU+MET+PHE+TYR+TRP)')
	surf_hyd_area = str(surf_hyd_all)

	# Charge
	no_pos = cmd.select('resname ARG+LYS and name ca')
	no_neg = cmd.select('resname GLU+ASP and name ca')
	charge = no_pos - no_neg
	charge = str(charge)
	no_neg = str(no_neg) 
	no_pos = str(no_pos) 

	# Number of SS
	no_ssx = cmd.select('CYS/SG and bound_to CYS/SG')
	no_ss = no_ssx/2
	no_ss = str(no_ss)


	# Number of long-range SS
	count = 0
	if no_ssx >= 2:
		cmd.do('''select long_ss, CYS/SG and bound_to CYS/SG''')
		cmd.do('''myspace={'l': []}''')
		cmd.iterate('long_ss', 'l.append(resv)', space=myspace)
		lss = myspace['l']
		cmd.do('''myspace={'d': []}''')

		count = 0
		for i in lss:

			t = []
			a = i

			cmd.select('d', '(not resid %s) and bound_to %s/SG' % (i, i))
			cmd.iterate('d', 't.append(resv)')

			if t:  # Check if t is not empty
				t_i = int("".join(map(str, t)))  # Safely convert elements to string and then to integer
				c = t_i - a

				if abs(c) >= 12:
					count += 1
				# No else needed; if t is empty or the condition is not met, we simply don't increment count

		count = count // 2  # Using integer division
		long_ss = str(count)
	else:
		long_ss = "N"



	stored.s = []
	

	# Secondary structure - percentage
	print(cmd.get_model("n. ca").atom)
	cmd.do('''s = [i.ss for i in cmd.get_model("n. ca").atom]''')
	cmd.iterate("s","stored.s=s")
	# print(cmd.get("s"))
	print(stored.s)

	alp = 100.0*s.count("H")/len(s)
	bet  = 100.0*s.count("S")/len(s)

	alp = str(alp)
	bet  = str(bet)
	
	
	# Salt-bridges (https://sourceforge.net/p/pymol/mailman/message/29141454/)
	cmd.do('''select roi, chain A''')
	cmd.do('''select positive, resn ARG+LYS and not name N+O''')
	cmd.do('''select negative, resn GLU+ASP and not name N+O''')

	cmd.do('''set h_bond_cutoff_center, 5.0''')
	cmd.do('''set h_bond_cutoff_edge, 5.0''')

	cmd.delete('saltbridges')
	cmd.do('''distance saltbridges, roi and negative, roi and positive, mode=2''')
	cmd.hide('label')

	cmd.do('''get_raw_distances''')
	cmd.do('''sb = get_raw_distances('saltbridges')''')
	sb = len(sb)
	sb = str(sb)
	
	cmd.do('''set h_bond_cutoff_center, 3.6''')
	cmd.do('''set h_bond_cutoff_edge, 3.6''')

	# Hydrogen bonds - all
	cmd.do('''distance H_bonds, roi, roi, quiet=1, mode=2, label=0, reset=1''')
	cmd.do('''get_raw_distances''')
	cmd.do('''hb = get_raw_distances('H_bonds')''')
	hb = len(hb)
	hb = str(hb)

	output.write(pdb + "\t" + no_aa + "\t" + no_gly + "\t" + no_ss + "\t" + long_ss + "\t" + charge + "\t" + sasa + "\t" + no_pos + "\t" + surf_pos + "\t" + surf_pos_area + "\t" + no_neg + "\t" + surf_neg + "\t" + surf_neg_area + "\t" + no_hyd + "\t" + surf_hyd + "\t" + surf_hyd_area + "\t" + alp + "\t" + bet + "\t" + sb + "\t" + hb + "\n")
	

	cmd.do('''delete all''')

output.close()

QStandardPaths: wrong permissions on runtime directory /run/user/1000/, 0755 instead of 0700


 Setting: fetch_path set to /home/mangus/Anubis/pdb.
PyMOL>run removealt.py
PyMOL>run findSurfaceResidues.py
PyMOL>run get_raw_distances.py
PyMOL>removealt
PyMOL>set dot_solvent, 1
 Setting: dot_solvent set to on.
PyMOL>set dot_density, 2
 Setting: dot_density set to 2.
PyMOL>findSurfaceResidues
Exposed atoms are selected in: exposed_atm_01
Exposed residues are selected in: exposed_res_01
No License File - For Evaluation Only (30 days remaining)
[<chempy.Atom object at 0x7fa9b013bee0>, <chempy.Atom object at 0x7fa9b013bf10>, <chempy.Atom object at 0x7fa9b013bf40>, <chempy.Atom object at 0x7fa9b013bf70>, <chempy.Atom object at 0x7fa9b013bfa0>, <chempy.Atom object at 0x7fa9b013bfd0>, <chempy.Atom object at 0x7fa9b018c040>, <chempy.Atom object at 0x7fa9b018c070>, <chempy.Atom object at 0x7fa9b018c0a0>, <chempy.Atom object at 0x7fa9b018c0d0>, <chempy.Atom object at 0x7fa9b018c100>, <chempy.Atom object at 0x7fa9b018c130>, <chempy.Atom object at 0x7fa9b018c160>, <chempy.Atom object at 0x7fa9

NameError: name 's' is not defined

In [1]:
import os


In [2]:
os.system('pymol PyMOL_Analysis.py')

QStandardPaths: wrong permissions on runtime directory /run/user/1000/, 0755 instead of 0700


 PyMOL(TM) 3.0.0 - Incentive Product
 Copyright (C) Schrodinger, LLC
 
 This Executable Build integrates and extends Open-Source PyMOL.
No License File - For Evaluation Only (30 days remaining)
 Detected OpenGL version 4.1. Shaders available.
 ShaderPrg-Error: vertex shader compilation failed.; name='bezier'
 ShaderPrg-Error-InfoLog:
0:1(10): error: GLSL 4.50 is not supported. Supported versions are: 1.10, 1.20, 1.30, 1.40, 1.50, 3.30, 4.00, 4.10, 1.00 ES, and 3.00 ES

 Detected GLSL version 4.10.
 OpenGL graphics engine:
  GL_VENDOR:   Microsoft Corporation
  GL_RENDERER: D3D12 (Intel(R) UHD Graphics 770)
  GL_VERSION:  4.1 (Compatibility Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
 ShaderPrg-Error: vertex shader compilation failed.; name='bezier'
 ShaderPrg-Error-InfoLog:
0:1(10): error: GLSL 4.50 is not supported. Supported versions are: 1.10, 1.20, 1.30, 1.40, 1.50, 3.30, 4.00, 4.10, 1.00 ES, and 3.00 ES


 Detected 24 CPU cores.  Enabled multithreaded rendering.
 Setting: fetch_path s

0